In [22]:
# 导入库
import pandas as pd
import numpy as np
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import torch

# 设置随机种子，确保结果可复现
np.random.seed(42)
torch.manual_seed(42)

# 读取数据
csv_path = 'train/data.csv'
data = pd.read_csv(csv_path)
print(f"数据集大小: {len(data)} 条记录")
data.head()

数据集大小: 840 条记录


,filename,number,xmin,ymin,xmax,ymax
0,hefei_3188.jpg,9575.9,42,70,315,115
1,hefei_3190.jpg,19652.0,57,50,339,107
2,hefei_3191.jpg,2262.7,19,85,337,143
3,hefei_3192.jpg,5702.0,48,54,305,96
4,hefei_3194.jpg,2566.0,99,59,261,98


In [23]:
import cv2
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd

# 创建保存处理后图像的目录
os.makedirs('train/Dataset_process', exist_ok=True)

def correct_distortion(image, k1=-0.05, k2=0.001, p1=0, p2=0):
    """
    矫正图像畸变
    参数:
        image: 输入图像 (numpy array)
        k1, k2: 径向畸变系数
        p1, p2: 切向畸变系数
    返回:
        矫正后的图像
    """
    h, w = image.shape[:2]
    # 相机内参矩阵
    K = np.array([
        [w/2, 0, w/2],
        [0, w/2, h/2],
        [0, 0, 1]
    ], dtype=np.float32)
    
    # 畸变系数
    D = np.array([k1, k2, p1, p2], dtype=np.float32)
    
    # 计算无畸变的映射关系
    map1, map2 = cv2.initUndistortRectifyMap(K, D, np.eye(3), K, (w, h), cv2.CV_32FC1)
    
    # 应用映射关系进行畸变矫正
    undistorted = cv2.remap(image, map1, map2, cv2.INTER_LINEAR)
    
    return undistorted

def enhance_contrast(image, clip_limit=2.0, tile_grid_size=(8, 8)):
    """
    增强图像对比度 (使用 CLAHE - 限制对比度的自适应直方图均衡化)
    参数:
        image: 输入图像 (numpy array)
        clip_limit: 对比度限制阈值
        tile_grid_size: 分块大小
    返回:
        增强对比度后的图像
    """
    # 转换为 LAB 色彩空间
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    
    # 应用 CLAHE 到 L 通道
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    cl = clahe.apply(l)
    
    # 合并通道并转回 BGR
    limg = cv2.merge((cl, a, b))
    enhanced = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    
    return enhanced

def reduce_reflection(image, gamma=0.7):
    """
    减少反光 (通过伽马校正)
    参数:
        image: 输入图像 (numpy array)
        gamma: 伽马值 (<1 增强暗部, >1 增强亮部)
    返回:
        减少反光后的图像
    """
    # 应用伽马校正
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    
    return cv2.LUT(image, table)

def process_image(input_path, output_path):
    """
    处理单张图像: 去光照、畸变矫正、对比度增强
    """
    # 读取图像
    image = cv2.imread(input_path)
    
    if image is None:
        print(f"警告: 无法读取图像 {input_path}")
        return False
    
    # 图像预处理流程
    try:
        # 1. 减少反光
        image = reduce_reflection(image)
        
        # 2. 畸变矫正
        image = correct_distortion(image)
        
        # 3. 增强对比度
        image = enhance_contrast(image)
        
        # 保存处理后的图像
        cv2.imwrite(output_path, image)
        return True
    except Exception as e:
        print(f"错误: 处理图像 {input_path} 时出错 - {str(e)}")
        return False

def batch_process_images(DATA_ROOT='train'):
    """
    批量处理目录中的所有图像
    """

    # 构建路径
    input_dir = f'{DATA_ROOT}/Dataset'
    output_dir = f'{DATA_ROOT}/Dataset_process'
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    image_files = [f for f in os.listdir(input_dir) 
                  if os.path.splitext(f)[1].lower() in image_extensions]
    
    total_images = len(image_files)
    success_count = 0
    
    print(f"开始处理 {total_images} 张图像...")
    
    for i, filename in enumerate(image_files):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)
        
        if process_image(input_path, output_path):
            success_count += 1
        
        # 打印进度
        if (i + 1) % 10 == 0 or (i + 1) == total_images:
            print(f"已处理: {i+1}/{total_images}, 成功: {success_count}")
    
    print(f"图像处理完成! 成功处理 {success_count}/{total_images} 张图像")
    return success_count

# 执行批量图像处理
success_count = batch_process_images()

# 更新数据集路径（如果需要使用处理后的图像进行训练）
# create_yolo_dataset 函数中的 image_dir 参数应改为 'Dataset_process'

开始处理 840 张图像...
已处理: 10/840, 成功: 10
已处理: 20/840, 成功: 20
已处理: 30/840, 成功: 30
已处理: 40/840, 成功: 40
已处理: 50/840, 成功: 50
已处理: 60/840, 成功: 60
已处理: 70/840, 成功: 70
已处理: 80/840, 成功: 80
已处理: 90/840, 成功: 90
已处理: 100/840, 成功: 100
已处理: 110/840, 成功: 110
已处理: 120/840, 成功: 120
已处理: 130/840, 成功: 130
已处理: 140/840, 成功: 140
已处理: 150/840, 成功: 150
已处理: 160/840, 成功: 160
已处理: 170/840, 成功: 170
已处理: 180/840, 成功: 180
已处理: 190/840, 成功: 190
已处理: 200/840, 成功: 200
已处理: 210/840, 成功: 210
已处理: 220/840, 成功: 220
已处理: 230/840, 成功: 230
已处理: 240/840, 成功: 240
已处理: 250/840, 成功: 250
已处理: 260/840, 成功: 260
已处理: 270/840, 成功: 270
已处理: 280/840, 成功: 280
已处理: 290/840, 成功: 290
已处理: 300/840, 成功: 300
已处理: 310/840, 成功: 310
已处理: 320/840, 成功: 320
已处理: 330/840, 成功: 330
已处理: 340/840, 成功: 340
已处理: 350/840, 成功: 350
已处理: 360/840, 成功: 360
已处理: 370/840, 成功: 370
已处理: 380/840, 成功: 380
已处理: 390/840, 成功: 390
已处理: 400/840, 成功: 400
已处理: 410/840, 成功: 410
已处理: 420/840, 成功: 420
已处理: 430/840, 成功: 430
已处理: 440/840, 成功: 440
已处理: 450/840, 成功: 450
已处理: 460/840

In [24]:
# 创建 YOLO 格式的数据集目录结构
def create_yolo_dataset(data, image_dir, output_dir, test_size=0.2):
    # 创建目录
    os.makedirs(f"{output_dir}/images/train", exist_ok=True)
    os.makedirs(f"{output_dir}/images/val", exist_ok=True)
    os.makedirs(f"{output_dir}/labels/train", exist_ok=True)
    os.makedirs(f"{output_dir}/labels/val", exist_ok=True)
    
    # 划分训练集和验证集
    train_files, val_files = train_test_split(data['filename'].unique(), test_size=test_size, random_state=42)
    
    # 处理每条数据
    for _, row in data.iterrows():
        filename = row['filename']
        xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
        
        # 计算 YOLO 格式的边界框坐标 (class x_center y_center width height)
        # 假设所有样本都是同一类 (class_id=0)
        img_width, img_height = 400, 297  # 根据您的图像尺寸调整
        x_center = (xmin + xmax) / (2 * img_width)
        y_center = (ymin + ymax) / (2 * img_height)
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height
        
        # 确定是训练集还是验证集
        if filename in train_files:
            split = 'train'
        else:
            split = 'val'
        
        # 复制图像
        src_img = os.path.join(image_dir, filename)
        dst_img = os.path.join(output_dir, f"images/{split}", filename)
        shutil.copy(src_img, dst_img)
        
        # 保存标签
        label_filename = filename.replace('.jpg', '.txt')
        dst_label = os.path.join(output_dir, f"labels/{split}", label_filename)
        with open(dst_label, 'w') as f:
            f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
    
    print(f"已创建 YOLO 数据集: {output_dir}")
    print(f"训练集大小: {len(train_files)}, 验证集大小: {len(val_files)}")

# 创建 YOLO 数据集
create_yolo_dataset(
    data=data,
    image_dir='train/Dataset_process',  # 图像所在目录
    output_dir='yolo_data',  # 输出 YOLO 格式数据的目录
    test_size=0.2
)

已创建 YOLO 数据集: yolo_data
训练集大小: 672, 验证集大小: 168


In [25]:
# 创建 dataset.yaml 配置文件
yaml_content = """
# 数据集路径
path: ../yolo_data  # 根路径
train: images/train  # 训练集路径
val: images/val  # 验证集路径

# 类别信息
nc: 1  # 类别数量
names: ['meter']  # 类别名称
"""

with open('dataset.yaml', 'w') as f:
    f.write(yaml_content)

print("已创建 dataset.yaml 配置文件")

已创建 dataset.yaml 配置文件


In [26]:
# 加载预训练模型
model = YOLO('yolov8s.pt')  # 可以选择 yolov8n.pt, yolov8m.pt 等不同大小的模型

# 训练模型
results = model.train(
    data='dataset.yaml',  # 数据集配置文件
    epochs=50,            # 训练轮次
    imgsz=400,            # 输入图像尺寸
    batch=16,             # 批次大小
    optimizer='Adam',     # 使用 Adam 优化器
    lr0=0.001,            # 初始学习率
    patience=20,          # 早停等待轮次
    project='yolov8_meter',  # 项目名称
    name='exp1',          # 实验名称
    pretrained=True,      # 使用预训练权重
    verbose=True          # 显示详细训练信息
)

Ultralytics 8.3.145  Python-3.11.5 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=400, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp111, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=20, perspective=0.0, plots=True, pose=12.0, pr

train: Scanning F:\桌面\digital_recongnition\MachineLearning\yolo_data\labels\train.cache... 672 images, 0 backgrounds, 0 corrupt: 100%|██████████| 672/672 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 468.3134.1 MB/s, size: 41.7 KB)


val: Scanning F:\桌面\digital_recongnition\MachineLearning\yolo_data\labels\val.cache... 168 images, 0 backgrounds, 0 corrupt: 100%|██████████| 168/168 [00:00<?, ?it/s]


Plotting labels to yolov8_meter\exp111\labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 416 train, 416 val
Using 8 dataloader workers
Logging results to yolov8_meter\exp111
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      1.98G     0.4844     0.2313     0.8951         16        416: 100%|██████████| 42/42 [00:06<00:00,  6.33it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:00<00:00,  7.41it/s]

                   all        168        168          1          1      0.995      0.898



50 epochs completed in 0.118 hours.
Optimizer stripped from yolov8_meter\exp111\weights\last.pt, 22.5MB
Optimizer stripped from yolov8_meter\exp111\weights\best.pt, 22.5MB

Validating yolov8_meter\exp111\weights\best.pt...
Ultralytics 8.3.145  Python-3.11.5 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:00<00:00,  6.42it/s]


                   all        168        168          1          1      0.995      0.899
Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to yolov8_meter\exp111


In [27]:
# 在验证集上评估模型
metrics = model.val()

# 打印评估结果
print(f"mAP@0.5: {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.p.mean():.4f}")  # 使用 .mean() 获取平均值
print(f"Recall: {metrics.box.r.mean():.4f}")    # 使用 .mean() 获取平均值


Ultralytics 8.3.145  Python-3.11.5 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 384.0180.1 MB/s, size: 41.9 KB)


val: Scanning F:\桌面\digital_recongnition\MachineLearning\yolo_data\labels\val.cache... 168 images, 0 backgrounds, 0 corrupt: 100%|██████████| 168/168 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:04<00:00,  2.44it/s]


                   all        168        168          1          1      0.995      0.897
Speed: 0.2ms preprocess, 4.7ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to yolov8_meter\exp1112
mAP@0.5: 0.9950
mAP@0.5:0.95: 0.8967
Precision: 1.0000
Recall: 0.9999


In [28]:
def predict_and_save(model, image_dir, data_root='train'):
    """
    使用训练好的YOLOv8模型对指定目录下的图像进行预测，
    并将结果保存为CSV文件，同时确保保留原始数据中的数值信息。
    """
    results = []
    output_csv=f'{data_root}/data_detect.csv'
    original_dir = f'{data_root}/data.csv'
    # 预先加载原始数据，以便快速查找数值信息
    original_data = pd.read_csv(original_dir)
    original_dict = {row['filename']: row['number'] for _, row in original_data.iterrows()}
    
    # 获取所有图像文件
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    total_images = len(image_files)
    print(f"找到 {total_images} 张图像，开始预测...")
    
    # 遍历图像进行预测
    for i, img_path in enumerate(image_files):
        full_path = os.path.join(image_dir, img_path)
        
        try:
            # 模型预测
            pred = model(full_path, imgsz=400, conf=0.5)
            
            # 处理预测结果
            if len(pred[0].boxes) > 0:
                # 获取置信度最高的边界框
                boxes = pred[0].boxes
                confidences = boxes.conf.cpu().numpy()
                max_idx = np.argmax(confidences)
                box = boxes[max_idx]
                
                # 获取边界框坐标并转换为整数
                xmin, ymin, xmax, ymax = box.xyxy[0].cpu().numpy().astype(int)
                
                # 从原始数据中获取对应的数值
                number = original_dict.get(img_path, None)
                
                results.append({
                    'filename': img_path,
                    'number': number,
                    'xmin': xmin,
                    'ymin': ymin,
                    'xmax': xmax,
                    'ymax': ymax
                })
                
                # 打印进度
                if (i + 1) % 10 == 0:
                    print(f"已处理: {i+1}/{total_images}")
            else:
                print(f"警告: 图像 {img_path} 未检测到电表区域")
                
        except Exception as e:
            print(f"错误: 处理图像 {img_path} 时出错 - {str(e)}")
    
    # 保存到CSV
    if results:
        pd.DataFrame(results).to_csv(output_csv, index=False)
        print(f"预测结果已保存到 {output_csv}，共 {len(results)} 条记录")
    else:
        print("警告: 没有生成任何预测结果")
    
    return results

# 对验证集进行预测
pred_results = predict_and_save(model, 'train/Dataset_process')

找到 840 张图像，开始预测...

WARNING imgsz=[400] must be multiple of max stride 32, updating to [416]


image 1/1 f:\\digital_recongnition\MachineLearning\train\Dataset_process\hefei_3188.jpg: 320x416 1 meter, 6.7ms
Speed: 0.8ms preprocess, 6.7ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 416)

WARNING imgsz=[400] must be multiple of max stride 32, updating to [416]
image 1/1 f:\\digital_recongnition\MachineLearning\train\Dataset_process\hefei_3190.jpg: 320x416 1 meter, 4.9ms
Speed: 0.7ms preprocess, 4.9ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 416)

WARNING imgsz=[400] must be multiple of max stride 32, updating to [416]
image 1/1 f:\\digital_recongnition\MachineLearning\train\Dataset_process\hefei_3191.jpg: 320x416 1 meter, 4.9ms
Speed: 1.0ms preprocess, 4.9ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 416)

WARNING imgsz=[400] must be multiple of max stride 32, updating to [416]
image 1/1 f:\\digital_recongnition\MachineLearning\train\Dataset_process\hefei_3192.jpg: 320x416 1 meter, 6.9ms
Speed: 1.1ms preprocess, 6.9ms inference, 

In [29]:
def calculate_iou(box1, box2):
    """计算两个边界框的IoU"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection
    
    return intersection / union if union > 0 else 0

def evaluate_performance(data_root='train'):
    """评估模型性能"""
    truth_csv=f'{data_root}/data.csv'
    pred_csv=f'{data_root}/data_detect.csv'
    truth_df = pd.read_csv(truth_csv)
    pred_df = pd.read_csv(pred_csv)
    
    # 合并数据
    merged_df = pd.merge(truth_df, pred_df, on='filename', suffixes=('_truth', '_pred'))
    
    # 计算IoU
    ious = []
    for _, row in merged_df.iterrows():
        truth_box = [row['xmin_truth'], row['ymin_truth'], row['xmax_truth'], row['ymax_truth']]
        pred_box = [row['xmin_pred'], row['ymin_pred'], row['xmax_pred'], row['ymax_pred']]
        ious.append(calculate_iou(truth_box, pred_box))
    
    # 计算准确率 (IoU >= 0.6的比例)
    accuracy = sum(iou >= 0.6 for iou in ious) / len(ious) * 100
    
    print(f"平均IoU: {np.mean(ious):.4f}")
    print(f"准确率 (IoU >= 0.6): {accuracy:.2f}%")
    
    return merged_df

# 评估模型
results_df = evaluate_performance()

平均IoU: 0.9290
准确率 (IoU >= 0.6): 100.00%


In [30]:
success_count = batch_process_images('test')

开始处理 168 张图像...
已处理: 10/168, 成功: 10
已处理: 20/168, 成功: 20
已处理: 30/168, 成功: 30
已处理: 40/168, 成功: 40
已处理: 50/168, 成功: 50
已处理: 60/168, 成功: 60
已处理: 70/168, 成功: 70
已处理: 80/168, 成功: 80
已处理: 90/168, 成功: 90
已处理: 100/168, 成功: 100
已处理: 110/168, 成功: 110
已处理: 120/168, 成功: 120
已处理: 130/168, 成功: 130
已处理: 140/168, 成功: 140
已处理: 150/168, 成功: 150
已处理: 160/168, 成功: 160
已处理: 168/168, 成功: 168
图像处理完成! 成功处理 168/168 张图像


In [35]:
model = YOLO("yolov8_meter/exp111/weights/best.pt")  # 加载最佳权重
pred_results = predict_and_save(model, 'test/Dataset_process','test')
results_df = evaluate_performance('test')

找到 168 张图像，开始预测...

WARNING imgsz=[400] must be multiple of max stride 32, updating to [416]
image 1/1 f:\\digital_recongnition\MachineLearning\test\Dataset_process\hefei_3191.jpg: 320x416 1 meter, 54.6ms
Speed: 2.2ms preprocess, 54.6ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 416)

WARNING imgsz=[400] must be multiple of max stride 32, updating to [416]
image 1/1 f:\\digital_recongnition\MachineLearning\test\Dataset_process\hefei_3195.jpg: 320x416 1 meter, 38.8ms
Speed: 2.1ms preprocess, 38.8ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 416)

WARNING imgsz=[400] must be multiple of max stride 32, updating to [416]
image 1/1 f:\\digital_recongnition\MachineLearning\test\Dataset_process\hefei_3198.jpg: 320x416 1 meter, 6.9ms
Speed: 0.8ms preprocess, 6.9ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 416)

WARNING imgsz=[400] must be multiple of max stride 32, updating to [416]
image 1/1 f:\\digital_recongnition\MachineLearning\test\Data